In [14]:
# creates the coregt1 folder and coregistrate every nii.gz files found in "original folder" to T1 file found in the patient folder
# creates the mni subfolder and coregistrate every nii.gz files found in "original folder" to MNI152 space

In [139]:
### DECLARATIONS AND IMPORTS ###
import os
from nipype.interfaces import fsl
from nipype.interfaces.dcm2nii import Dcm2nii
from nipype.interfaces.freesurfer import MRIConvert

import pandas as pd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import glob
import dicom2nifti
import re
from nilearn import plotting
import fnmatch

%matplotlib notebook

In [140]:
# PARAMETERS TO CHANGE
# change before run: Select the original folder where T1_masked_with_aseg.nii.gz is present and the images are in "original" folder
DATA_DIR = "/home/nraresearch/Downloads/test/crpp11_visit1_02062020"
PATH_GLOBAL_CSV_POSTOP = os.path.join(DATA_DIR, '..','..','values_all_postop.csv')
PATH_GLOBAL_CSV_CT_MRI = os.path.join(DATA_DIR, '..','..','values_all.csv')

In [141]:
# CONSTANTS
os.chdir(DATA_DIR)
bins = 256
PATH_MNI_BRAINMASK = ref_img_mni = '/home/nraresearch/research/perfusion_integral/data/crpps_perfusionintegral/ALLCRPP_T1_CBF_MOCO_T1MASKS/mni152_brainmask.nii.gz'
PATH_T1_BRAINMASK = os.path.join(DATA_DIR, "t1_masked_with_aseg.nii.gz")
PATH_MASK_PENUMBRA = os.path.join(DATA_DIR, "coregt1", "coregt1_mask_penumbra_bl.nii.gz")


# specify all the possible sequence names that the algorihtm should iterate on
# more names can be listed. Only exact matches will be processed
PERFUSION_FILES = [ "MTT", "rBF", "rBV", "TMAX" , "TTP", "tMIP", "tMIPS"]
MASK_FILES = ["mask_core_v1","mask_penumbra","mask_penumbra_bl", "mask_penumbra_v1", "mask_penumbra_bl_4-6"]
DWI_FILES = ["ADC","IVIM_ADC", "IVIM_TRACEW_1","IVIM_TRACEW_2","IVIM_TRACEW_3","IVIM_TRACEW_4","IVIM_TRACEW_5","IVIM_TRACEW_B5", "B1000","B1000_NATIVE","Bxxxx" ]
NIFTI_FILES = ["FLAIR"]
BOLD_FILES = ["CVRmap"]
SELECTED_TO_ANALYSIS = ["rBF", "rBV", "TMAX" ,"B1000","ADC","IVIM_TRACEW_5","IVIM_ADC","CVRmap","FLAIR"]

In [142]:
### HELPER FUNCTIONS ###

# save to hemisphere masks to nifti
def save_nifti(np_array, filename, output_folder,  original_img ):
    """
    Saves nifti file
    np_array: input 3D numpy matrix without header
    filename: example: "cbf" then the output name will be cbf.nii.gz
    original img: helper image that passes the header and affine parameters
    output folder:
    """
    original_img = nb.load(PATH_MASK_MRI)

    filepath = OUTPUT_FOLDER + '/' + filename + '.nii.gz'
    
    if not os.path.isfile(filepath):
        # array_data = np.arange(24, dtype=np.int16).reshape((2, 3, 4))
        # affine = np.diag([1, 2, 3, 1])
        nii_img = nb.nifti1.Nifti1Image(np_array, original_img.affine , header=original_img.header)
        nb.save(nii_img, filepath)
        print('Image has been saved to {}'.format(filepath))
    else:
        print("Image already exists!")
        
# run flirt with default 6 dof
def run_flirt(path_infile, 
              out_file,
              path_reference, 
              dof, 
              bins = 256, 
              out_matrix_file = 'transformation_matrix.mat', 
              cost = 'corratio',
              #interp = 'nearestneighbour'):
              interp = 'trilinear'):
    
    #out_file_cbf = 'mni_'+os.path.basename(path_infile)[:-7] + '_to_' + os.path.basename(path_reference)[:-7]+'_dof'+ str(dof) + '.nii.gz'

    print("flirt coregistration for file: {}".format(out_file))

    flt = fsl.FLIRT(bins=bins, 
                    cost_func=cost, 
                    interp = interp, 
                    in_file = path_infile,
                    reference = path_reference,
                    out_file = out_file,
                    dof = dof,
                    out_matrix_file = os.path.join( os.path.dirname(os.path.realpath(out_file)), out_matrix_file),
                    output_type = "NIFTI_GZ")

    if not os.path.isfile(out_file):
        res = flt.run()
        print("Coregistration has saved to: {}".format(out_file))
    else:
        print("Flirt coregistration already exists!")
    
    return out_file

        
# function for BET brain extraction
def run_bet(input_file_path):
    """
    input:
    input_file_path 
    
    output:
    path_outfile_bet: path of the resulted brain extracted file
    path_betmask: full file path of the binary brain extraction mask
    """
    btr = fsl.BET()
    OUTPUT_FOLDER = os.path.dirname(input_file_path)

    _out_file_bet = 'bet_' + os.path.basename(input_file_path)[:-7] + '.nii.gz'
    path_outfile_bet = os.path.join(OUTPUT_FOLDER, _out_file_bet)
    
    if not os.path.isfile(path_outfile_bet):
        result = btr.run(in_file= input_file_path, out_file=path_outfile_bet, frac=0.7 , mask = True)
    else:
        print("Bet file already exists")
    
    # path of the binary mask file
    _filename_mask = 'bet_' + os.path.basename(input_file_path)[:-7] + '_mask.nii.gz'
    path_betmask = os.path.join(OUTPUT_FOLDER, _filename_mask)
    return path_outfile_bet, path_betmask

def save_nifti(np_array, filename, output_folder,  original_img ):
    """
    Saves nifti file
    np_array: input 3D numpy matrix without header
    filename: example: "cbf" then the output name will be cbf.nii.gz
    original img: helper image that passes the header and affine parameters
    output folder:
    """
    original_img = nb.load(original_img)

    filepath = output_folder + '/' + filename + '.nii.gz'
    
    # if not os.path.isfile(filepath):
    # array_data = np.arange(24, dtype=np.int16).reshape((2, 3, 4))
    # affine = np.diag([1, 2, 3, 1])
    nii_img = nb.nifti1.Nifti1Image(np_array, original_img.affine , header=original_img.header)
    nb.save(nii_img, filepath)
    print('Image has been saved to {}'.format(filepath))
    #else:
    #   print("Image already exists!")
        
# event handler for scrolling 3D images
def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)
        
def multi_slice_viewer(volume):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()
    ax.volume = volume
    ax.index = volume.shape[0] // 2
    ax.imshow(volume[ax.index])
    fig.canvas.mpl_connect('key_press_event', process_key)

def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()

def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    ax.images[0].set_array(volume[ax.index])

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[0]
    ax.images[0].set_array(volume[ax.index])

In [143]:
# mri convert all the files from "original" folder and save to "resliced" folder
os.chdir(DATA_DIR)
mc = MRIConvert()
path = "./original" 
try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

dir_resliced_name = "resliced"
path_dir_resliced = os.path.join(os.getcwd(),"../"+ dir_resliced_name)
if not os.path.isdir(path_dir_resliced):
    os.mkdir(path_dir_resliced)
                                 
for i_img in PERFUSION_FILES + MASK_FILES + DWI_FILES + NIFTI_FILES + BOLD_FILES: 
    path_out_file = os.path.join(os.getcwd(), "../resliced/r_" + i_img + ".nii.gz")
    if not os.path.isfile(path_out_file):
        try:
            mc.inputs.in_file = os.path.join(os.getcwd(), i_img + '.nii.gz')
            mc.inputs.out_file = path_out_file
            mc.inputs.out_type = 'niigz'
            mc.inputs.reslice_like = PATH_T1_BRAINMASK
            res_r = mc.run()
            
        except:
            print("Failure at processing file %s", path_out_file)
    else:
        print("File %s already exist " % path_out_file)

Current working directory: /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_MTT.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_rBF.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_rBV.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_TMAX.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_TTP.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_tMIP.nii.gz already exist 
Failure at processing file %s /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_tMIPS.nii.gz
Failure at processing file %s /home/nraresearch/Downloads/test/crpp11_visit1_02062020/original/../resliced/r_mask_core_v1.ni

In [144]:
# coregister the not perfusion nifti files NIFTI_FILES = ["ADC","IVIM_ADC, IVIM_TRACEW_B5"] 
os.chdir(DATA_DIR)
path = os.path.join(DATA_DIR, dir_resliced_name)
try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

# create coregt1 folder to store the t1 space coregistered nifti files
dir_coreg_name = "coregt1"
path_dir_coregt= os.path.join(os.getcwd(),"../"+dir_coreg_name)
if not os.path.isdir(path_dir_coregt):
    os.mkdir(path_dir_coregt)

# coregistrate B1000 and store transformation matrix
tag_base_file = "B1000"
# tag_base_file = "IVIM_TRACEW_5"

PATH_BASE_DWI = os.path.abspath("./r_" +tag_base_file +".nii.gz")

if os.path.isfile(PATH_BASE_DWI):
    path_matrix_file = os.path.join( os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + tag_base_file +"_2_t1matrix.mat")

    print("Baseline DWI file is {}".format(PATH_BASE_DWI))
    path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + tag_base_file  + ".nii.gz")
    PATH_CBF_2_BET_T1= run_flirt(path_infile = PATH_BASE_DWI,
                                 out_file=path_out_file,
                                 path_reference = PATH_T1_BRAINMASK,
                                 out_matrix_file= path_matrix_file,
                                 dof = 6)

    # iterate through perfusion maps
    for i_img in DWI_FILES : 
        path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img + ".nii.gz")
        if not os.path.isfile(path_out_file):
            try:
                path_infile = os.path.join(os.getcwd(),"r_" + i_img + '.nii.gz')
                print("Processing {}".format(i_img))
                applyxfm = fsl.preprocess.ApplyXFM()
                applyxfm.inputs.in_file = path_infile
                applyxfm.inputs.in_matrix_file = path_matrix_file
                applyxfm.inputs.out_file = path_out_file
                applyxfm.inputs.reference = PATH_T1_BRAINMASK
                applyxfm.inputs.apply_xfm = True
                result = applyxfm.run()            
            except:
                print("Failure at processing file %s" % path_out_file)
        else:
            print("File %s already exist " % path_out_file)
else:
    print("B1000 baseline file does not exist to calculate the transformation matrix for perfusion files. Choose a new baseline file or place the B1000 file in the original folder")

    
# coregistrate rBV and store transformation matrix
tag_base_file = "rBV"
PATH_BASE_PERF = os.path.abspath("./r_" +tag_base_file +".nii.gz")

if os.path.isfile(PATH_BASE_PERF):

    path_matrix_perf = os.path.join( os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + tag_base_file +"_2_t1matrix.mat")

    print("Baseline perfusion file is {}".format(PATH_BASE_PERF))
    path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + tag_base_file  + ".nii.gz")
    PATH_CBF_2_BET_T1= run_flirt(path_infile = PATH_BASE_PERF,
                                 out_file=path_out_file,
                                 path_reference = PATH_T1_BRAINMASK,
                                 out_matrix_file= path_matrix_perf,
                                 dof = 6)

    # iterate through perfusion maps
    for i_img in PERFUSION_FILES + MASK_FILES : 
        path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img + ".nii.gz")
        if not os.path.isfile(path_out_file):
            try:
                path_infile = os.path.join(os.getcwd(),"r_" + i_img + '.nii.gz')
                print("Processing {}".format(i_img))
                applyxfm = fsl.preprocess.ApplyXFM()
                applyxfm.inputs.in_file = path_infile
                applyxfm.inputs.in_matrix_file = path_matrix_perf
                applyxfm.inputs.out_file = path_out_file
                applyxfm.inputs.reference = PATH_T1_BRAINMASK
                applyxfm.inputs.apply_xfm = True
                result = applyxfm.run()            
            except:
                print("Failure at processing file %s" % i_img)
        else:
            print("File %s already exist " % path_out_file)
else:
    print("rBV baseline file does not exist to calculate the transformation matrix for perfusion files. Choose a new baseline file or place the B1000 file in the original folder")

# coregistrate bold, and other nifti files in different spaces (T2 , FLAIR etc)

for i_img in NIFTI_FILES:
    try:
        path_infile = os.path.join(os.getcwd(),"r_" + i_img + '.nii.gz')
        # path_matrix_file = os.path.join( os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img +"_2_t1matrix.mat")
        path_matrix_file = os.path.join( os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img +"_2_t1matrix.mat")

        "inv_coreg_T1_BOLD"
        print("Coregistration of {} to T1".format(i_img))
        path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img  + ".nii.gz")
        PATH_CBF_2_BET_T1= run_flirt(path_infile = path_infile,
                                     out_file=path_out_file,
                                     path_reference = PATH_T1_BRAINMASK,
                                     out_matrix_file= path_matrix_file,
                                     dof = 6)
    except:
        print("File %s could not be processed" % i_img)

Current working directory: /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced
Baseline DWI file is /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/r_B1000.nii.gz
flirt coregistration for file: /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_B1000.nii.gz
Flirt coregistration already exists!
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_ADC.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_IVIM_ADC.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_IVIM_TRACEW_1.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_IVIM_TRACEW_2.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/../coregt1/coregt1_IVIM_TRACEW_3.nii.gz already exist 
File /home/nraresearch/Downloads

In [145]:
# registrate bold file with inverse transformation matrix
for i_img in BOLD_FILES:
    #try:
    path_infile = os.path.join(os.getcwd(),"r_" + i_img + '.nii.gz')
    path_matrix_file_bold = os.path.join( os.getcwd(), "inv_coreg_T1_BOLD.txt")
    
    print("Coregistration of {} to T1".format(i_img))
    path_out_file = os.path.join(os.getcwd(), "../" + dir_coreg_name  + "/" + dir_coreg_name +"_" + i_img  + ".nii.gz")
    try:
        print("Processing {}".format(i_img))
        applyxfm = fsl.preprocess.ApplyXFM()
        applyxfm.inputs.in_file = path_infile
        applyxfm.inputs.in_matrix_file = path_matrix_file_bold
        applyxfm.inputs.out_file = path_out_file
        applyxfm.inputs.reference = PATH_T1_BRAINMASK
        applyxfm.inputs.apply_xfm = True
        result = applyxfm.run()   
    

    except:
        print("File %s could not be processed" % i_img)


os.chdir("..")

Coregistration of CVRmap to T1
Processing CVRmap
220408-21:16:45,214 nipype.interface INFO:
	 stderr 2022-04-08T21:16:45.214650:Could not open matrix file /home/nraresearch/Downloads/test/crpp11_visit1_02062020/resliced/inv_coreg_T1_BOLD.txt
220408-21:16:45,221 nipype.interface INFO:
	 stderr 2022-04-08T21:16:45.221800:matrix multiplication: incompatible matrix dimensions: 4x4 and 0x0
File CVRmap could not be processed


In [146]:
# coregister all the NIFTI_FILES to MNI wiht transformation matrix between t1 and MNI152
os.chdir(DATA_DIR)
path = "./" + dir_coreg_name
try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

dir_mni_name = "mni"
path_dir_mni= os.path.join(os.getcwd(),"../"+dir_mni_name)
if not os.path.isdir(path_dir_mni):
    os.mkdir(path_dir_mni)


# coregistrate T1 to MNI and store transofrmation matrix
tag_base_file = "t1_masked_with_aseg"
path_matrix_mni_file = os.path.join( os.getcwd(), "../" + dir_mni_name  + "/" + tag_base_file +"_2_mnimatrix.mat")

print("Path of T1 file to be coregistered to MNI is {}".format(PATH_T1_BRAINMASK))
path_out_file = os.path.join(os.getcwd(), "../" + dir_mni_name  + "/" + dir_mni_name +"_" + tag_base_file  + ".nii.gz")
PATH_CBF_2_BET_T1= run_flirt(path_infile = PATH_T1_BRAINMASK,
                             out_file=path_out_file,
                             path_reference = PATH_MNI_BRAINMASK,
                             out_matrix_file= path_matrix_mni_file,
                             dof = 12)

# iterate through all files and apply the transformation matrix calculated above between the patient's t1  and mni space
for i_img in PERFUSION_FILES+ MASK_FILES + DWI_FILES + NIFTI_FILES + BOLD_FILES : 
    path_out_file = os.path.join(os.getcwd(), "../" + dir_mni_name  + "/" + dir_mni_name +"_" + i_img + ".nii.gz")
    if not os.path.isfile(path_out_file):
        try:
            path_infile = os.path.join(os.getcwd(),"coregt1_" + i_img + '.nii.gz')
            print("Processing {}".format(i_img))
            applyxfm = fsl.preprocess.ApplyXFM()
            applyxfm.inputs.in_file = path_infile
            applyxfm.inputs.in_matrix_file = path_matrix_mni_file
            applyxfm.inputs.out_file = path_out_file
            #applyxfm.inputs.reference = PATH_T1_BRAINMASK
            applyxfm.inputs.reference = PATH_MNI_BRAINMASK
            applyxfm.inputs.apply_xfm = True
            result = applyxfm.run()            
        except:
            print("Failure at processing file %s" % path_out_file)
    else:
        print("File %s already exist " % path_out_file)

Current working directory: /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1
Path of T1 file to be coregistered to MNI is /home/nraresearch/Downloads/test/crpp11_visit1_02062020/t1_masked_with_aseg.nii.gz
flirt coregistration for file: /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_t1_masked_with_aseg.nii.gz
Flirt coregistration already exists!
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_MTT.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_rBF.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_rBV.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_TMAX.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_TTP.nii.gz already exist 
File /home/nraresearch/Downloads/test/crpp11_visit1_02062020/coregt1/../mni/mni_

In [147]:
def get_name_of_folder(a_path):
    without_extra_slash = os.path.normpath(a_path)
    last_part = os.path.basename(without_extra_slash)
    return last_part

def get_name_of_patient(a_path):
    a_path = os.path.join(a_path, '..')
    without_extra_slash = os.path.normpath(a_path)
    last_part = os.path.basename(without_extra_slash)
    return last_part

def get_name_of_visit(a_path):
    without_extra_slash = os.path.normpath(a_path)
    last_part = os.path.basename(without_extra_slash)
    pattern = "_(.*?)\_"
    substring = re.search(pattern, last_part).group(1)
    return substring

In [153]:
### calculate the values
#  read the coregistered CBF file
if os.path.exists(PATH_TMAX):
    PATH_TMAX = os.path.join(os.path.dirname(PATH_MASK_PENUMBRA),'coregt1_TMAX.nii.gz' )
    vol_tmax = nb.load(PATH_TMAX)
    np_vol_tmax = vol_tmax.get_fdata()

if os.path.exists(PATH_RBV):
    PATH_RBV = os.path.join(os.path.dirname(PATH_MASK_PENUMBRA),'coregt1_rBV.nii.gz' )
    vol_rbv = nb.load(PATH_RBV)
    np_vol_rbv = vol_rbv.get_fdata()

if os.path.exists(PATH_RBF):
    PATH_RBF = os.path.join(os.path.dirname(PATH_MASK_PENUMBRA),'coregt1_rBF.nii.gz' )
    vol_rbf = nb.load(PATH_RBF)
    np_vol_rbf = vol_rbf.get_fdata()

PATH_MASK_PENUMBRA = os.path.join(DATA_DIR, "coregt1", "coregt1_mask_penumbra_bl.nii.gz")
# load the penumbra masks
if os.path.exists(PATH_MASK_PENUMBRA):
    vol_mask = nb.load(PATH_MASK_PENUMBRA)
    vol_mask.shape
    np_vol_mask = vol_mask.get_fdata()
    np_vol_mask[np_vol_tmax < 6] = 0
    np_vol_mask[np_vol_rbv < 0] = 0
    np_vol_mask[np_vol_rbf < 0]  = 0
    
PATH_MASK_PENUMBRA46 = os.path.join(DATA_DIR, "coregt1", "coregt1_mask_penumbra_bl_4-6.nii.gz")
# load the penumbra masks
if os.path.exists(PATH_MASK_PENUMBRA46):
    vol_mask_penumbra46 = nb.load(PATH_MASK_PENUMBRA46)
    vol_mask.shape
    np_vol_mask_p46 = vol_mask_penumbra46.get_fdata()
    np_vol_mask_p46[np_vol_tmax < 6] = 0
    np_vol_mask_p46[np_vol_rbv < 0] = 0
    np_vol_mask_p46[np_vol_rbf < 0]  = 0    

PATH_MASK_CORE = os.path.join(DATA_DIR, "coregt1", "coregt1_mask_core_v1.nii.gz")
if os.path.exists(PATH_MASK_CORE):
    vol_mask_core = nb.load(PATH_MASK_CORE)
    vol_mask.shape
    np_vol_mask_core = vol_mask_core.get_fdata()
    np_vol_mask_core[np_vol_tmax < 6] = 0
    np_vol_mask_core[np_vol_rbv < 0] = 0
    np_vol_mask_core[np_vol_rbf < 0]  = 0

columns = ['patient','visit']
values = [get_name_of_patient(DATA_DIR), get_name_of_visit(DATA_DIR) ]
values_p46 = [get_name_of_patient(DATA_DIR), get_name_of_visit(DATA_DIR) ]
values_core = [get_name_of_patient(DATA_DIR), get_name_of_visit(DATA_DIR) ]

# index = [get_name_of_folder(DATA_DIR) + str(os.path.basename(PATH_CBF_PERF)[4:-7])] 
index = [get_name_of_folder(DATA_DIR)] 

for i_seq in SELECTED_TO_ANALYSIS:    
    list_calculated = [i_seq+'_mean',
                       i_seq+'_std',
                       i_seq+'_min',
                       i_seq+'_max',
                       i_seq+'_median',
                       i_seq+'_q1',
                       i_seq+'_q3']
    columns = columns + list_calculated
    
    path_infile = os.path.join(DATA_DIR,"coregt1", "coregt1_" + i_seq + '.nii.gz')

    if os.path.isfile(path_infile):
        vol = nb.load(path_infile)
        np_vol = vol.get_fdata()
        
        if os.path.exists(PATH_MASK_PENUMBRA):
            # apply left hemisphere mask on flirt_cbf_to_bett1
            np_vol_masked = np.zeros(np_vol_mask.shape)
            np.putmask(np_vol_masked, np_vol_mask, np_vol)
            roi = np_vol_masked[np_vol_mask>0.5]       
            seq_values = [np.mean(roi), np.std(roi), np.min(roi), np.max(roi),np.median(roi),np.percentile(roi,25),np.percentile(roi,75) ]
            print(seq_values)
            
        if os.path.exists(PATH_MASK_PENUMBRA46):
            # apply left hemisphere mask on flirt_cbf_to_bett1
            np_vol_masked_p46  = np.zeros(np_vol_mask_p46 .shape)
            np.putmask(np_vol_masked_p46 , np_vol_mask_p46 , np_vol)
            roi_p46  = np_vol_masked_p46 [np_vol_mask_p46 >0.5]
            seq_values_p46 = [np.mean(roi_p46), np.std(roi_p46), np.min(roi_p46), np.max(roi_p46),np.median(roi_p46),np.percentile(roi_p46,25),np.percentile(roi_p46,75) ]
            print(seq_values_p46)
            
        if os.path.exists(PATH_MASK_CORE):
            # apply left hemisphere mask on flirt_cbf_to_bett1
            np_vol_masked_core = np.zeros(np_vol_mask_core.shape)
            np.putmask(np_vol_masked_core , np_vol_mask_core , np_vol)
            roi_core  = np_vol_masked_core [np_vol_mask_core >0.5]      
            seq_values_core = [np.mean(roi_core ), np.std(roi_core ), np.min(roi_core ), np.max(roi_core ),np.median(roi_core ),np.percentile(roi_core ,25),np.percentile(roi_core ,75) ]
            print(seq_values_core )
            
    else:
        # seq_values = np.full([1, len(list_calculated)],1)
        # seq_values = np.full([1, len(list_calculated)], 0)
        if os.path.exists(PATH_MASK_PENUMBRA):
            seq_values = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan ]
        if os.path.exists(PATH_MASK_PENUMBRA46):
            seq_values_p46 = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan ]
        if os.path.exists(PATH_MASK_CORE):
            seq_values_core = [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan ]

    # values to the sequence
    if os.path.exists(PATH_MASK_PENUMBRA):
        values = values + seq_values
        df_penumbra= pd.DataFrame(index=index, columns=columns)
        df_penumbra.loc[index[0]] = values
        PATH_CSV_LOCAL = os.path.join(DATA_DIR, get_name_of_folder(DATA_DIR)+'_penumbra.csv')
        df_penumbra.to_csv(PATH_CSV_LOCAL, index=True, mode='w+', sep = ',')
        #df_penumbra.to_csv(PATH_GLOBAL_CSV_POSTOP, mode='a', header=False)
        df_penumbra.to_csv(PATH_GLOBAL_CSV_CT_MRI, mode='a', header=False)

    if os.path.exists(PATH_MASK_PENUMBRA46):
        values_p46 = values_p46 + seq_values_p46
        df_p46= pd.DataFrame(index=index, columns=columns)
        df_p46.loc[index[0]] = values_p46
        PATH_CSV_LOCAL_p46 = os.path.join(DATA_DIR, get_name_of_folder(DATA_DIR)+'._p46.csv')
        df_p46.to_csv(PATH_CSV_LOCAL_p46, index=True, mode='w+', sep = ',')

    if os.path.exists(PATH_MASK_CORE):
        values_core = values_core + seq_values_core
        df_core= pd.DataFrame(index=index, columns=columns)
        df_core.loc[index[0]] = values_core
        PATH_CSV_LOCAL_core = os.path.join(DATA_DIR, get_name_of_folder(DATA_DIR)+'_core.csv')
        df_core.to_csv(PATH_CSV_LOCAL_core, index=True, mode='w+', sep = ',')



[15.891674420325238, 10.144580790026001, 2.8008806705474854, 71.75411224365234, 11.918086051940918, 8.306972026824951, 21.41731834411621]
[2.479785262639343, 2.3878041286707847, 0.0015633292496204376, 20.387252807617188, 1.7190444469451904, 0.8612765073776245, 3.3096368312835693]
[39.941800322165015, 13.342622273117586, 6.837252616882324, 88.81098937988281, 38.44941711425781, 30.643566131591797, 47.4523983001709]
[14.098797057561923, 8.895571038750298, 1.4603441953659058, 50.0540885925293, 11.727184295654297, 7.885516166687012, 18.574377059936523]
[2.594302949173639, 0.9200442708602836, 0.5963921546936035, 4.496624946594238, 2.911224603652954, 1.5870878100395203, 3.244931936264038]
[13.643343064731308, 9.172713224859637, 1.0165128707885742, 51.686370849609375, 11.099312782287598, 7.40958571434021, 18.0566349029541]
[2603.6130443135903, 919.2083003748232, 358.2047119140625, 4030.53955078125, 2948.704833984375, 1611.5526733398438, 3261.0462646484375]
